# Cellular economic - TD on resource allocation
January 2024, by Léa Wagner, Olivier Borkowski and Matthieu Jules

In [3]:
# Installing the required packages for the whole TD
pip install -r requirements.txt

## Model 1 - cell-free

To begin with, we consider an extremely simplified representation of a cell:

<img src='240109_Model1.png' width=50%/>

As depicted above, the cell is made of only five coarse-grained molecular components: 
- an internalized substrate $s_i$
- a species $a$ representing energy
- a pool of ribosomes $r$ 
- a pool of housekeeping proteins $h$
- a pool of enzymes for metabolism $e_{m1}$

The total energy available to the cell must be divided between the tasks of producing $r$, $h$ and $e_{m1}$. This is represented in the model by the parameters $f_R$, $f_H$ and $f_{em1}$, which represent the proportion of energy allocated to the production of $r$, $h$ and $e_{m1}$ respectively.

Ribosomes produce $r$, $h$ and $e_{m1}$ according to a Michaelis Menten law: 
$g_{r}(a) = \frac{f_x * a * v_r}{(f_x * a) + k_r}$ where f_x is the proportion of energy allocated to the production of the considered species.

Similarly, the metabolic enzymes $e_{m1}$ enable the regeneration of energy $a$ according to a Michaelis Menten law: 
$g_{em1}(s_i) = \frac{s_i * v_{em1}}{s_i + k_{em1}}$

### Questions
1.1) Write the system of differential equations describing this model.

1.2) Using the graph below, we will describe the effects of variations in the proportions of each species produced.

&emsp; a) Describe the effects on the system of a variation in the __ribosome fraction__. What happens when no ribosomes are neoformed? What happens when only ribosomes are produced? 

&emsp; b) Describe the effects on the system of a variation in the __metabolic enzymes fraction__. What happens when no metabolic enzymes are neoformed? What happens when only metabolic enzymes are produced? 

&emsp; c) Describe the effects on the system of a variation in the __housekeeping proteins fraction__. What happens when no housekeeping proteins are neoformed? What happens when only housekeeping proteins are produced? Interpret the biological significance. 

In [27]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
from ipywidgets import interact_manual, widgets
from IPython.display import display

def MM(x, vr, kr, fi):
    return (x * fi * vr) / (x * fi + kr)

def derivs_mod0(t, y, params):
    Si, A, R, H, Em1 = y
    dSi_dt = - MM(Si, params['v_a'], params['k_a'],1) * Em1
    dA_dt = MM(Si, params['v_a'], params['k_a'],1) * Em1 - MM(A, params['v_r'], params['k_r'], params['f_em1']) * R - MM(A, params['v_r'], params['k_r'], params['f_h']) * R - MM(A, params['v_r'], params['k_r'], params['f_r']) * R
    dR_dt = MM(A, params['v_r'], params['k_r'], params['f_r']) * R
    dH_dt = MM(A, params['v_r'], params['k_r'], params['f_h']) * R
    dEm1_dt = MM(A, params['v_r'], params['k_r'], params['f_em1']) * R
    return np.array([dSi_dt, dA_dt, dR_dt, dH_dt, dEm1_dt])

def simulate_mod0(params, duration, initial_conditions):
    y0 = np.array(initial_conditions)
    ode_result = integrate.solve_ivp(lambda t, y: derivs_mod0(t, y, params),
                                     (0, duration),
                                     y0,
                                     dense_output=True,
                                     rtol=1e-6, atol=1e-9)
    return ode_result.t, ode_result.y

def plot_dynamic(f_em1, f_r, f_h):
    # Fix values for v_a, k_a, v_r, k_r, and duration
    v_a, k_a, v_r, k_r, duration = 10, 1, 10, 1, 2
    
    # Check if the sum of f_em1, f_r, and f_h is equal to 1
    if not np.isclose(f_em1 + f_r + f_h, 1, rtol=1e-5):
        print("Error: Sum of f_em1, f_r, and f_h must be equal to 1.")
        return
    
    model_params = {'v_a': v_a, 'k_a': k_a, 'v_r': v_r, 'k_r': k_r, 'f_em1': f_em1, 'f_h': f_h, 'f_r': f_r}
    initial_conditions = [5, 0, 1, 1, 1]  # You can adjust the initial conditions as needed
    
    t, results = simulate_mod0(model_params, duration, initial_conditions)
    
    # Plotting the results
    plt.figure(figsize=(12, 8))
    
    # Plot each variable separately with specified colors
    plt.plot(t, results[0], label='si', color='orange')
    plt.plot(t, results[1], label='a', color='brown')
    plt.plot(t, results[2], label='r', color='lightgreen')
    plt.plot(t, results[3], label='h', color='red')
    plt.plot(t, results[4], label='em1', color='darkgreen') 
    
    plt.xlabel('Time')
    plt.ylabel('Concentration')
    plt.title('Simulation of the model 0 - cell-free')
    
    # Set y-axis limits to [0, 8]
    plt.ylim([0, 8])
    
    # Add legend based on the variables in your model
    plt.legend()
    
    # Annotate the plot with parameter values
    params_text = '\n'.join([f'{key}: {value}' for key, value in model_params.items()])
    plt.text(1.05, 0.5, params_text, transform=plt.gca().transAxes, fontsize=12, verticalalignment='center')
    
    # Annotate the plot with initial composition
    init_comp_text = f'Initial Composition:\nsi: {initial_conditions[0]}, a: {initial_conditions[1]}, r: {initial_conditions[2]}, h: {initial_conditions[3]}, em1: {initial_conditions[4]}'
    plt.text(1.05, 0.2, init_comp_text, transform=plt.gca().transAxes, fontsize=12, verticalalignment='center')
    
    plt.show()

# Create FloatSliders with initial values
f_em1 = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.2)
f_r = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.4)
f_h = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.4)

# Use interact_manual with the plotting function and sliders
interact_manual(plot_dynamic, f_em1=f_em1, f_r=f_r, f_h=f_h)


interactive(children=(FloatSlider(value=0.2, description='f_em1', max=1.0), FloatSlider(value=0.4, description…

<function __main__.plot_dynamic(f_em1, f_r, f_h)>

1.3) Using the graph below, what species can be absent from the initial conditions without the system collapsing? Which species are essential to initialize the model?

In [26]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
from ipywidgets import interact_manual, widgets
from IPython.display import display

def MM(x, vr, kr, fi):
    return (x * fi * vr) / (x * fi + kr)

def derivs_mod0(t, y, params):
    Si, A, R, H, Em1 = y
    dSi_dt = - MM(Si, params['v_a'], params['k_a'],1) * Em1
    dA_dt = MM(Si, params['v_a'], params['k_a'],1) * Em1 - MM(A, params['v_r'], params['k_r'], params['f_em1']) * R - MM(A, params['v_r'], params['k_r'], params['f_h']) * R - MM(A, params['v_r'], params['k_r'], params['f_r']) * R
    dR_dt = MM(A, params['v_r'], params['k_r'], params['f_r']) * R
    dH_dt = MM(A, params['v_r'], params['k_r'], params['f_h']) * R
    dEm1_dt = MM(A, params['v_r'], params['k_r'], params['f_em1']) * R
    return np.array([dSi_dt, dA_dt, dR_dt, dH_dt, dEm1_dt])

def simulate_mod0(params, duration, initial_conditions):
    y0 = np.array(initial_conditions)
    ode_result = integrate.solve_ivp(lambda t, y: derivs_mod0(t, y, params),
                                     (0, duration),
                                     y0,
                                     dense_output=True,
                                     rtol=1e-6, atol=1e-9)
    return ode_result.t, ode_result.y

def plot_dynamic(initial_Si, initial_A, initial_R, initial_Em1):
    # Fix values for v_a, k_a, v_r, k_r, f_em1, f_r, f_h, and duration
    v_a, k_a, v_r, k_r, f_em1, f_r, f_h, duration = 10, 1, 10, 1, 0.30, 0.50, 0.20, 2
    
    # Check if the sum of f_em1, f_r, and f_h is equal to 1
    if not np.isclose(f_em1 + f_r + f_h, 1, rtol=1e-5):
        print("Error: Sum of f_em1, f_r, and f_h must be equal to 1.")
        return
    
    model_params = {'v_a': v_a, 'k_a': k_a, 'v_r': v_r, 'k_r': k_r, 'f_em1': f_em1, 'f_h': f_h, 'f_r': f_r}
    initial_conditions = [initial_Si, initial_A, initial_R, 0, initial_Em1]  # H is set to 0
    
    t, results = simulate_mod0(model_params, duration, initial_conditions)
    
    # Plotting the results
    plt.figure(figsize=(12, 8))
    
    # Plot each variable separately with specified colors
    plt.plot(t, results[0], label='si', color='orange')
    plt.plot(t, results[1], label='a', color='brown')
    plt.plot(t, results[2], label='r', color='lightgreen')
    plt.plot(t, results[3], label='h', color='red')
    plt.plot(t, results[4], label='em1', color='darkgreen') 
    
    plt.xlabel('Time')
    plt.ylabel('Concentration')
    plt.title('Simulation of the model 0 - cell-free')
    
    # Set y-axis limits to [0, 8]
    plt.ylim([0, 8])
    
    # Add legend based on the variables in your model
    plt.legend()
    
    # Annotate the plot with parameter values
    params_text = '\n'.join([f'{key}: {value}' for key, value in model_params.items()])
    plt.text(1.05, 0.5, params_text, transform=plt.gca().transAxes, fontsize=12, verticalalignment='center')
    
    # Annotate the plot with initial composition
    init_comp_text = f'Initial composition:\nsi: {initial_conditions[0]}, a: {initial_conditions[1]}, r: {initial_conditions[2]}, h: {initial_conditions[3]}, em1: {initial_conditions[4]}'
    plt.text(1.05, 0.2, init_comp_text, transform=plt.gca().transAxes, fontsize=12, verticalalignment='center')
    
    plt.show()

# Create FloatSliders with initial values
initial_Si = widgets.FloatSlider(min=0, max=5, step=0.1, value=5)
initial_A = widgets.FloatSlider(min=0, max=5, step=0.1, value=0)
initial_R = widgets.FloatSlider(min=0, max=5, step=0.1, value=1)
initial_Em1 = widgets.FloatSlider(min=0, max=5, step=0.1, value=0)

# Use interact_manual with the plotting function and sliders
interact_manual(plot_dynamic, initial_Si=initial_Si, initial_A=initial_A, initial_R=initial_R, initial_Em1=initial_Em1)


interactive(children=(FloatSlider(value=5.0, description='initial_Si', max=5.0), FloatSlider(value=0.0, descri…

<function __main__.plot_dynamic(initial_Si, initial_A, initial_R, initial_Em1)>

1.4) Describe the different types of resources present in this system (for catabolism or anabolism).

1.5) Why does the concentration of some species never decrease? Which classical biological phenomenon is not considered by this simplified model? 

1.6) In all the above cases, the system eventually reaches a stage where nothing more is produced. Why? 

## Model 2 - <i>in vivo

Let's consider now a slightly more complicated system to represent a cell _in vivo_. Unlike cell-free, this cell is constrained by a membrane. 
1. The cell must internalize its substrate ($s$) from the surrounding environment using transport enzymes ($e_t$).
2. This compartmentalization gives rise to density constraints. Based on experimental data, we assume that the cell maintains a constant mass density and volume. That means the cell mass (equal to the sum of all components) is proportional to the growth rate.
    
<img src='240109_Model2.png' width=50%/>
    
As depicted above, the cell is made of the following species: 
- an external substrate $s$
- an internalized substrate $s_i$
- a species $a$ representing both protein precursors and energy
- a pool of ribosomes $r$ 
- a pool of housekeeping proteins $h$
- a pool of enzymes for metabolism $e_{m1}$
- a pool of enzymes for transport $e_t$
    
The actions of $r$, $e_{m1}$ and $e_t$ are modelled by a Michaelis-Menten function as before.

### Questions

2.1) Write the system of differential equations describing this model. Add a term to account for dilution that take the growth rate into account.

2.2) Using the graph below, we will describe the effects of variations in the proportions of each species produced.

&emsp; a) Describe the effects on the system of a variation in the __ribosome fraction__. What happens when no ribosomes are neoformed? What happens when only ribosomes are produced? 

&emsp; b) Describe the effects on the system of a variation in the __metabolic enzymes fraction__. What happens when no metabolic enzymes are neoformed? What happens when only metabolic enzymes are produced? 

&emsp; c) Describe the effects on the system of a variation in the __housekeeping proteins fraction__. What happens when no housekeeping proteins are neoformed? What happens when only housekeeping proteins are produced?

In [28]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
from ipywidgets import interact_manual, widgets
from IPython.display import display

def MM(x, vr, kr, fi):
    return (x * fi * vr) / (x * fi + kr)

def derivs_mod0(t, y, params):
    S, Si, A, R, H, Em1, Et, µ = y
    dS_dt = - MM(S, params['v_et'], params['k_et'],1) * Et
    dSi_dt = MM(S, params['v_et'], params['k_et'],1) * Et - MM(Si, params['v_a'], params['k_a'],1) * Em1 - µ * Si
    dA_dt  = MM(Si, params['v_a'], params['k_a'],1) * Em1 -  MM(A, params['v_r'], params['k_r'], params['f_em1']) * R -  MM(A, params['v_r'], params['k_r'], params['f_h']) * R -  MM(A, params['v_r'], params['k_r'], params['f_r']) * R -  MM(A, params['v_r'], params['k_r'], params['f_et']) * R  - µ * A
    dR_dt  =  MM(A, params['v_r'], params['k_r'], params['f_r']) * R  - µ * R
    dH_dt  =  MM(A, params['v_r'], params['k_r'], params['f_h']) * R  - µ * H
    dEm1_dt=  MM(A, params['v_r'], params['k_r'], params['f_em1']) * R  - µ * Em1
    dEt_dt =  MM(A, params['v_r'], params['k_r'], params['f_et']) * R  - µ * Et
    µ = (dSi_dt + dA_dt + dR_dt + dH_dt + dEm1_dt+ dEt_dt)/(Si + A + R + H + Em1 + Et)
    return np.array([dS_dt, dSi_dt, dA_dt, dR_dt, dH_dt, dEm1_dt, dEt_dt, µ])

def simulate_mod0(params, duration, initial_conditions):
    y0 = np.array(initial_conditions)
    ode_result = integrate.solve_ivp(lambda t, y: derivs_mod0(t, y, params),
                                     (0, duration),
                                     y0,
                                     dense_output=True,
                                     rtol=1e-6, atol=1e-9)
    return ode_result.t, ode_result.y

def plot_dynamic(f_em1, f_r, f_h, f_et):

    # Fix values for v_a, k_a, v_r, k_r, and duration
    v_a, k_a, v_r, k_r, v_et, k_et, duration = 10, 1, 10, 1, 7, 1, 100
    #v_a, k_a, v_r, k_r, v_et, k_et, duration = 5, 1, 3, 1, 7, 1, 100
    
    # Check if the sum of f_em1, f_r, and f_h is equal to 1
    if not np.isclose(f_em1 + f_r + f_h + f_et, 1, rtol=1e-5):
        print("Error: Sum of f_em1, f_r, f_h and f_et must be equal to 1.")
        return
    
    model_params = {'v_a': v_a, 'k_a': k_a, 'v_r': v_r, 'k_r': k_r, 'v_et': v_et, 'k_et': k_et, 'f_em1': f_em1, 'f_h': f_h, 'f_r': f_r, 'f_et': f_et}
    initial_conditions = [500, 0, 0, 1, 1, 1, 1, 0]  # You can adjust the initial conditions as needed
    
    t, results = simulate_mod0(model_params, duration, initial_conditions)
    
    # Plotting the results
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.suptitle('Simulation of the model 1 - in vivo')
    
    # Plot each variable separately with specified colors
    ax1.plot(t, results[1], label='si', color='orange')
    ax1.plot(t, results[2], label='a', color='pink')
    ax1.plot(t, results[3], label='r', color='lightgreen')
    ax1.plot(t, results[4], label='h', color='red')
    ax1.plot(t, results[5], label='em1', color='darkgreen') 
    ax1.plot(t, results[6], label='et', color='blue')
    ax1.set(xlabel='Time', ylabel='Concentration')

    ax2.plot(t, results[7], label='µ',color='black')
    ax2.set(xlabel='Time', ylabel='µ')

    # Set y-axis limits to [0, 8]
    ax2.set_ylim([0, 3])
    
    # Add legend based on the variables in your model
    ax1.legend()
    ax2.legend()
    
    # Annotate the plot with parameter values
    params_text = '\n'.join([f'{key}: {value}' for key, value in model_params.items()])
    plt.text(1.05, 0.7, params_text, transform=plt.gca().transAxes, fontsize=12, verticalalignment='center')
    
    # Annotate the plot with initial composition
    init_comp_text = f'Initial composition:\ns: {initial_conditions[0]}, si: {initial_conditions[1]}, a: {initial_conditions[2]},\nr: {initial_conditions[3]}, h: {initial_conditions[4]}, em1: {initial_conditions[5]}, et: {initial_conditions[6]}, µ: {initial_conditions[7]}'
    ax1.text(1.05, 0.3, init_comp_text, transform=plt.gca().transAxes, fontsize=12, verticalalignment='center')
    
    plt.subplots_adjust(wspace=0.5)
    plt.show()
    
# Create FloatSliders with initial values
f_em1 = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.1)
f_r = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.1)
f_h = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.7)
f_et = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.1)

# Use interact_manual with the plotting function and sliders
interact_manual(plot_dynamic, f_em1=f_em1, f_r=f_r, f_h=f_h, f_et=f_et)

interactive(children=(FloatSlider(value=0.1, description='f_em1', max=1.0), FloatSlider(value=0.1, description…

<function __main__.plot_dynamic(f_em1, f_r, f_h, f_et)>

2.3) Why does the concentration of R, H em1 and et are constant at some point, while there is still energy?

2.4) Find an expression for the cell growth rate $\mu$ (think mass conservation and combine the differential equations).

2.5) Using the graph below, see how $\mu$ changes when you increase the richness of the substrate $S$.

In [30]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
from ipywidgets import interact_manual, widgets
from IPython.display import display

def MM(x, vr, kr, fi):
    return (x * fi * vr) / (x * fi + kr)

def derivs_mod0(t, y, params):
    S, Si, A, R, H, Em1, Et, µ = y
    dS_dt = - MM(S, params['v_et'], params['k_et'],1) * Et
    dSi_dt = MM(S, params['v_et'], params['k_et'],1) * Et - MM(Si, params['v_a'], params['k_a'],1) * Em1 - µ * Si
    dA_dt  = MM(Si, params['v_a'], params['k_a'],1) * Em1 -  MM(A, params['v_r'], params['k_r'], params['f_em1']) * R -  MM(A, params['v_r'], params['k_r'], params['f_h']) * R -  MM(A, params['v_r'], params['k_r'], params['f_r']) * R -  MM(A, params['v_r'], params['k_r'], params['f_et']) * R  - µ * A
    dR_dt  =  MM(A, params['v_r'], params['k_r'], params['f_r']) * R  - µ * R
    dH_dt  =  MM(A, params['v_r'], params['k_r'], params['f_h']) * R  - µ * H
    dEm1_dt=  MM(A, params['v_r'], params['k_r'], params['f_em1']) * R  - µ * Em1
    dEt_dt =  MM(A, params['v_r'], params['k_r'], params['f_et']) * R  - µ * Et
    µ = (dSi_dt + dA_dt + dR_dt + dH_dt + dEm1_dt+ dEt_dt)/(Si + A + R + H + Em1 + Et)
    return np.array([dS_dt, dSi_dt, dA_dt, dR_dt, dH_dt, dEm1_dt, dEt_dt, µ])

def simulate_mod0(params, duration, initial_conditions):
    y0 = np.array(initial_conditions)
    ode_result = integrate.solve_ivp(lambda t, y: derivs_mod0(t, y, params),
                                     (0, duration),
                                     y0,
                                     dense_output=True,
                                     rtol=1e-6, atol=1e-9)
    return ode_result.t, ode_result.y

def plot_dynamic(initial_f_em1=0.3, initial_f_r=0.4, initial_f_h=0.1, initial_f_et=0.2, initial_S=1000):
    # Fix values for v_a, k_a, v_r, k_r, and duration
    v_a, k_a, v_r, k_r, v_et, k_et, duration = 10, 1, 10, 1, 7, 1, 100  # CHANGE HERE

    # Check if the sum of initial_f_em1, initial_f_r, initial_f_h, and initial_f_et is equal to 1
    if not np.isclose(initial_f_em1 + initial_f_r + initial_f_h + initial_f_et, 1, rtol=1e-5):
        print("Error: Sum of initial_f_em1, initial_f_r, initial_f_h, and initial_f_et must be equal to 1.")
        return

    model_params = {'v_a': v_a, 'k_a': k_a, 'v_r': v_r, 'k_r': k_r, 'v_et': v_et, 'k_et': k_et,
                    'f_em1': initial_f_em1, 'f_h': initial_f_h, 'f_r': initial_f_r, 'f_et': initial_f_et}
    initial_conditions = [initial_S, 0, 0, 1, 1, 1, 1, 0]  # S is now a variable

    def simulate_and_plot(S):
        initial_conditions[0] = S  # Update S in the initial conditions
        t, results = simulate_mod0(model_params, duration, initial_conditions)

        # Plotting the results
        fig, (ax1, ax2) = plt.subplots(1,2)
        fig.suptitle('Simulation of the model 1 - in vivo')

        # Plot each variable separately with specified colors
        ax1.plot(t, results[1], label='si', color='orange')
        ax1.plot(t, results[2], label='a', color='pink')
        ax1.plot(t, results[3], label='r', color='lightgreen')
        ax1.plot(t, results[4], label='h', color='red')
        ax1.plot(t, results[5], label='em1', color='darkgreen') 
        ax1.plot(t, results[6], label='et', color='blue')
        ax1.set(xlabel='Time', ylabel='Concentration')

        ax2.plot(t, results[7], label='µ',color='black')
        ax2.set(xlabel='Time', ylabel='µ')

        # Add legend based on the variables in your model
        ax1.legend()
        ax2.legend()

        # Annotate the plot with parameter values
        params_text = '\n'.join([f'{key}: {value}' for key, value in model_params.items()])
        plt.text(1.05, 0.7, params_text, transform=plt.gca().transAxes, fontsize=12, verticalalignment='center')
        
        # Annotate the plot with initial composition
        init_comp_text = f'Initial composition:\ns: {initial_conditions[0]}, si: {initial_conditions[1]}, a: {initial_conditions[2]},\nr: {initial_conditions[3]}, h: {initial_conditions[4]}, em1: {initial_conditions[5]}, et: {initial_conditions[6]}, µ: {initial_conditions[7]}'
        ax1.text(1.05, 0.3, init_comp_text, transform=plt.gca().transAxes, fontsize=12, verticalalignment='center')
        
        # Adjust the y axis
        ax2.set_ylim([0, 1.5])
        plt.subplots_adjust(wspace=0.5)
        plt.show()

    # Create FloatSlider for S with initial value
    S_slider = widgets.FloatSlider(min=0, max=100, step=5, value=initial_S)

    # Use interact_manual with the plotting function and slider
    interact_manual(simulate_and_plot, S=S_slider)

# Call the function with the initial parameters
plot_dynamic()

interactive(children=(FloatSlider(value=100.0, description='S', step=5.0), Button(description='Run Interact', …

2.6) Using the graph below, find the optimal ratios f_E, f_R to maximise the growth rate?

In [31]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
from ipywidgets import interact_manual, widgets
from IPython.display import display

def MM(x, vr, kr, fi):
    return (x * fi * vr) / (x * fi + kr)

def derivs_mod0(t, y, params):
    S, Si, A, R, H, Em1, Et, µ = y
    dS_dt = - MM(S, params['v_et'], params['k_et'],1) * Et
    dSi_dt = MM(S, params['v_et'], params['k_et'],1) * Et - MM(Si, params['v_a'], params['k_a'],1) * Em1 - µ * Si
    dA_dt  = MM(Si, params['v_a'], params['k_a'],1) * Em1 -  MM(A, params['v_r'], params['k_r'], params['f_em1']) * R -  MM(A, params['v_r'], params['k_r'], params['f_h']) * R -  MM(A, params['v_r'], params['k_r'], params['f_r']) * R -  MM(A, params['v_r'], params['k_r'], params['f_et']) * R  - µ * A
    dR_dt  =  MM(A, params['v_r'], params['k_r'], params['f_r']) * R  - µ * R
    dH_dt  =  MM(A, params['v_r'], params['k_r'], params['f_h']) * R  - µ * H
    dEm1_dt=  MM(A, params['v_r'], params['k_r'], params['f_em1']) * R  - µ * Em1
    dEt_dt =  MM(A, params['v_r'], params['k_r'], params['f_et']) * R  - µ * Et
    µ = (dSi_dt + dA_dt + dR_dt + dH_dt + dEm1_dt+ dEt_dt)/(Si + A + R + H + Em1 + Et)
    return np.array([dS_dt, dSi_dt, dA_dt, dR_dt, dH_dt, dEm1_dt, dEt_dt, µ])

def simulate_mod0(params, duration, initial_conditions):
    y0 = np.array(initial_conditions)
    ode_result = integrate.solve_ivp(lambda t, y: derivs_mod0(t, y, params),
                                     (0, duration),
                                     y0,
                                     dense_output=True,
                                     rtol=1e-6, atol=1e-9)
    return ode_result.t, ode_result.y

def plot_dynamic(f_em1, f_r, f_h, f_et):

    # Fix values for v_a, k_a, v_r, k_r, and duration
    v_a, k_a, v_r, k_r, v_et, k_et, duration = 10, 1, 10, 1, 7, 1, 100
    #v_a, k_a, v_r, k_r, v_et, k_et, duration = 5, 1, 3, 1, 7, 1, 100
    
    # Check if the sum of f_em1, f_r, and f_h is equal to 1
    if not np.isclose(f_em1 + f_r + f_h + f_et, 1, rtol=1e-5):
        print("Error: Sum of f_em1, f_r, f_h and f_et must be equal to 1.")
        return
    
    model_params = {'v_a': v_a, 'k_a': k_a, 'v_r': v_r, 'k_r': k_r, 'v_et': v_et, 'k_et': k_et, 'f_em1': f_em1, 'f_h': f_h, 'f_r': f_r, 'f_et': f_et}
    initial_conditions = [1000, 0, 0, 1, 1, 1, 1, 0]  # You can adjust the initial conditions as needed
    
    t, results = simulate_mod0(model_params, duration, initial_conditions)
    
    # Plotting the results
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.suptitle('Simulation of the model 1 - in vivo')
    
    # Plot each variable separately with specified colors
    ax1.plot(t, results[1], label='si', color='orange')
    ax1.plot(t, results[2], label='a', color='pink')
    ax1.plot(t, results[3], label='r', color='lightgreen')
    ax1.plot(t, results[4], label='h', color='red')
    ax1.plot(t, results[5], label='em1', color='darkgreen') 
    ax1.plot(t, results[6], label='et', color='blue')
    ax1.set(xlabel='Time', ylabel='Concentration')

    ax2.plot(t, results[7], label='µ',color='black')
    ax2.set(xlabel='Time', ylabel='µ')

    # Set y-axis limits to [0, 8]
    ax2.set_ylim([0, 3])
    
    # Add legend based on the variables in your model
    ax1.legend()
    ax2.legend()
    
    # Annotate the plot with parameter values
    params_text = '\n'.join([f'{key}: {value}' for key, value in model_params.items()])
    plt.text(1.05, 0.7, params_text, transform=plt.gca().transAxes, fontsize=12, verticalalignment='center')
    
    # Annotate the plot with initial composition
    init_comp_text = f'Initial composition:\ns: {initial_conditions[0]}, si: {initial_conditions[1]}, a: {initial_conditions[2]},\nr: {initial_conditions[3]}, h: {initial_conditions[4]}, em1: {initial_conditions[5]}, et: {initial_conditions[6]}, µ: {initial_conditions[7]}'
    ax1.text(1.05, 0.3, init_comp_text, transform=plt.gca().transAxes, fontsize=12, verticalalignment='center')
    
    plt.subplots_adjust(wspace=0.5)
    plt.show()
    
# Create FloatSliders with initial values
f_em1 = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.1)
f_r = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.1)
f_h = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.7)
f_et = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.1)

# Use interact_manual with the plotting function and sliders
interact_manual(plot_dynamic, f_em1=f_em1, f_r=f_r, f_h=f_h, f_et=f_et)

interactive(children=(FloatSlider(value=0.1, description='f_em1', max=1.0), FloatSlider(value=0.1, description…

<function __main__.plot_dynamic(f_em1, f_r, f_h, f_et)>

## Model 3 - _in vivo_ with two metabolic pathways

Let's consider now a slightly more complicated system to represent a cell _in vivo_. In cells, several metabolic pathways are encoded in the genome and can be activated depending on the environment. Here we show :
- an expensive but highly efficient metabolic pathway (em1) 
- an inexpensive but not very efficient metabolic pathway (em2)
    
<img src='240109_Model3.png' width=60%/>
    
As depicted above, the cell is made of the following species: 
- an external substrate $s$
- an internalized substrate $s_i$
- a species $a$ representing both protein precursors and energy
- a pool of ribosomes $r$ 
- a pool of housekeeping proteins $h$
- a pool of enzymes for metabolism $e_{m1}$ and $e_{m2}$
- a pool of enzymes for transport $e_t$
    
The actions of $r$, $e_{m1}$, $e_{m2}$ and $e_t$ are modelled by a Michaelis-Menten function as before.

3.1) Which of the metabolic pathways could correspond to 𝑒𝑚1 and 𝑒𝑚2 based on their characteristics?

3.2) Using the graph below, explore the circumstances in which 𝑒𝑚1 is more favorable than 𝑒𝑚2 and vice versa by varying the parameters.

In [34]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
from ipywidgets import interact_manual, widgets
from IPython.display import display

def MM(x, vr, kr, fi):
    return (x * fi * vr) / (x * fi + kr)

def derivs_mod0(t, y, params):
    S, Si, A, R, H, Em1, Em2, Et, µ = y
    dS_dt = - MM(S, params['v_et'], params['k_et'],1) * Et
    dSi_dt = MM(S, params['v_et'], params['k_et'],1) * Et - MM(Si, params['v_a'], params['k_a'],1) * Em1 - MM(Si, params['v_a2'], params['k_a2'],1) * Em2 - µ * Si
    dA_dt  = MM(Si, params['v_a'], params['k_a'],1) * Em1 + MM(Si, params['v_a2'], params['k_a2'],1) * Em2 -  MM(A, params['v_r'], params['k_r'], params['f_em1']) * R -  MM(A, params['v_r'], params['k_r'], params['f_em2'])  * R -  MM(A, params['v_r'], params['k_r'], params['f_h']) * R -  MM(A, params['v_r'], params['k_r'], params['f_r']) * R -  MM(A, params['v_r'], params['k_r'], params['f_et']) * R  - µ * A
    dR_dt  =  MM(A, params['v_r'], params['k_r'], params['f_r']) * R  - µ * R
    dH_dt  =  MM(A, params['v_r'], params['k_r'], params['f_h']) * R  - µ * H
    dEm1_dt=  MM(A, params['v_r'], params['k_r'], params['f_em1']) * 0.7 *R  - µ * Em1
    dEm2_dt=  MM(A, params['v_r'], params['k_r'], params['f_em2']) * 2 * R  - µ * Em2
    dEt_dt =  MM(A, params['v_r'], params['k_r'], params['f_et']) * R  - µ * Et
    µ = (dSi_dt + dA_dt + dR_dt + dH_dt + dEm1_dt + dEm2_dt + dEt_dt)/(Si + A + R + H + Em1 + Em2 + Et)
    return np.array([dS_dt, dSi_dt, dA_dt, dR_dt, dH_dt, dEm1_dt, dEm2_dt, dEt_dt, µ])

def simulate_mod0(params, duration, initial_conditions):
    y0 = np.array(initial_conditions)
    ode_result = integrate.solve_ivp(lambda t, y: derivs_mod0(t, y, params),
                                     (0, duration),
                                     y0,
                                     dense_output=True,
                                     rtol=1e-6, atol=1e-9)
    return ode_result.t, ode_result.y

def plot_dynamic(f_em1, f_em2, f_r, f_h, f_et):
    # Fix values for v_a, k_a, v_r, k_r, and duration
    v_a, k_a, v_a2, k_a2, v_r, k_r, v_et, k_et, duration = 10, 1, 5, 1, 10, 1, 7, 1, 100
        
    # Check if the sum of f_em1, f_r, and f_h is equal to 1
    if not np.isclose(f_em1 + f_em2 + f_r + f_h + f_et, 1, rtol=1e-5):
        print("Error: Sum of f_em1, f_r, f_h and f_et must be equal to 1.")
        return
    
    model_params = {'v_a': v_a, 'k_a': k_a, 'v_a2': v_a2, 'k_a2': k_a2, 'v_r': v_r, 'k_r': k_r, 'v_et': v_et, 'k_et': k_et, 'f_em1': f_em1, 'f_em2': f_em2, 'f_h': f_h, 'f_r': f_r, 'f_et': f_et}
    initial_conditions = [1000, 0, 0, 1, 1, 1, 1, 1, 0]  # You can adjust the initial conditions as needed
    
    t, results = simulate_mod0(model_params, duration, initial_conditions)
    
    # Plotting the results
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.suptitle('Simulation of the model 2 - in vivo')
    
    # Plot each variable separately with specified colors
    ax1.plot(t, results[1], label='Si', color='orange')
    ax1.plot(t, results[2], label='A', color='pink')
    ax1.plot(t, results[3], label='R', color='lightgreen')
    ax1.plot(t, results[4], label='H', color='red')
    ax1.plot(t, results[5], label='Em1', color='darkgreen') 
    ax1.plot(t, results[6], label='Em2', color='black')
    ax1.plot(t, results[7], label='Et', color='blue')
    ax1.set(xlabel='Time', ylabel='Concentration')

    ax2.plot(t, results[8], label='µ',color='black')
    ax2.set(xlabel='Time', ylabel='µ')

    # Set y-axis limits
    ax2.set_ylim([0, 3])
    
    # Add legend based on the variables in your model
    ax1.legend()
    ax2.legend()
    
    # Annotate the plot with parameter values
    params_text = '\n'.join([f'{key}: {value}' for key, value in model_params.items()])
    plt.text(1.05, 0.7, params_text, transform=plt.gca().transAxes, fontsize=12, verticalalignment='center')
    
    # Annotate the plot with initial composition
    init_comp_text = f'\nInitial composition:\ns: {initial_conditions[0]}, si: {initial_conditions[1]}, a: {initial_conditions[2]},\nr: {initial_conditions[3]}, h: {initial_conditions[4]}, em1: {initial_conditions[5]}, et: {initial_conditions[6]}, µ: {initial_conditions[7]}'
    ax1.text(1.05, 0.3, init_comp_text, transform=plt.gca().transAxes, fontsize=12, verticalalignment='center')
    
    plt.subplots_adjust(wspace=0.5)
    plt.show()

# Create FloatSliders with initial values
f_em1 = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.3)
f_em2 = widgets.FloatSlider(min=0, max=1, step=0.1, value=0)
f_r = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.1)
f_h = widgets.FloatSlider(min=0, max=1, step=0.1, value=0)
f_et = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.6)

# Use interact_manual with the plotting function and sliders
interact_manual(plot_dynamic, f_em1=f_em1, f_em2=f_em2, f_r=f_r, f_h=f_h, f_et=f_et)

interactive(children=(FloatSlider(value=0.3, description='f_em1', max=1.0), FloatSlider(value=0.0, description…

<function __main__.plot_dynamic(f_em1, f_em2, f_r, f_h, f_et)>